Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1]:
from random import choices

import lab9_lib

### It is applied a GA with Simulated Annealing-based Local Search.

In [3]:
from random import choices, randint, random, uniform
from math import exp

class GeneticAlgorithm:
    def __init__(self, problem, genome_size=1000, population_size=50, generations=100, mutation_rate=0.1, local_search_iterations=10, convergence_threshold=5):
        """
        Initialize the Genetic Algorithm parameters.

        Args:
        - problem: The optimization problem to solve.
        - genome_size: The size of the binary genome for each individual.
        - population_size: The number of individuals in the population.
        - generations: The maximum number of generations to run the algorithm.
        - mutation_rate: The probability of mutation for each bit in the genome.
        - local_search_iterations: The number of iterations for the local search algorithm.
        - convergence_threshold: The threshold for consecutive generations without improvement to terminate the algorithm.
        """
        self.problem = problem
        self.genome_size = genome_size
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        self.local_search_iterations = local_search_iterations
        self.convergence_threshold = convergence_threshold

    def run(self):
        """
        Run the Genetic Algorithm.

        Returns:
        - best_individual_after_local: The best individual after local search.
        """
        # Initialize the population
        population = [choices([0, 1], k=self.genome_size) for _ in range(self.population_size)]

        best_individual_after_local = None
        consecutive_no_improvement = 0

        for generation in range(self.generations):
            # Evaluate the fitness of the population
            fitness_scores = [self.problem(individual) for individual in population]
            best_individual = population[fitness_scores.index(max(fitness_scores))]

            # Apply local search to the best individual
            local_search = LocalSearch(self.problem, best_individual, iterations=self.local_search_iterations)
            local_search.run()
            best_individual_after_local = local_search.best_individual
            fitness_after_local = local_search.best_fitness

            # If fitness after local search improves, reset the no improvement counter
            if fitness_after_local > self.problem(best_individual):
                consecutive_no_improvement = 0
            else:
                consecutive_no_improvement += 1

            # If no improvement for a certain number of consecutive generations, terminate the algorithm
            if consecutive_no_improvement >= self.convergence_threshold:
                break

            # If fitness after local search improves, replace an individual in the population
            if fitness_after_local > self.problem(best_individual):
                index_to_replace = fitness_scores.index(min(fitness_scores))
                population[index_to_replace] = best_individual_after_local

            # Update the population using crossover and mutation
            new_population = self.crossover(population)
            self.mutate(new_population)
            population = new_population

        # Return the best individual after all generations
        return best_individual_after_local

    def crossover(self, population):
        """
        Apply crossover to generate new individuals from the existing population.

        Args:
        - population: The current population.

        Returns:
        - new_population: The new population after crossover.
        """
        new_population = []

        for _ in range(self.population_size // 2):
            # Tournament selection of parents
            parent1 = choices(population)[0]
            parent2 = choices(population)[0]

            # Two-point crossover
            crossover_points = sorted([randint(0, self.genome_size - 1) for _ in range(2)])

            child1 = parent1[:crossover_points[0]] + parent2[crossover_points[0]:crossover_points[1]] + parent1[crossover_points[1]:]
            child2 = parent2[:crossover_points[0]] + parent1[crossover_points[0]:crossover_points[1]] + parent2[crossover_points[1]:]

            new_population.extend([child1, child2])

        return new_population

    def mutate(self, population):
        """
        Apply mutation to each individual in the population.

        Args:
        - population: The current population.
        """
        for i in range(len(population)):
            for j in range(self.genome_size):
                if random() < self.mutation_rate:
                    population[i][j] = 1 - population[i][j]


class LocalSearch:
    def __init__(self, problem, initial_solution, iterations=10):
        """
        Initialize the Local Search algorithm parameters.

        Args:
        - problem: The optimization problem to solve.
        - initial_solution: The initial solution for local search.
        - iterations: The number of iterations for the local search.
        """
        self.problem = problem
        self.genome_size = len(initial_solution)
        self.best_individual = initial_solution
        self.best_fitness = problem(self.best_individual)
        self.iterations_without_improvement = 0
        self.iterations = iterations

    def run(self):
        """Run the Local Search algorithm."""
        for _ in range(self.iterations):
            neighbor = self.generate_neighbor(self.best_individual)
            neighbor_fitness = self.problem(neighbor)

            if neighbor_fitness > self.best_fitness:
                self.best_individual = neighbor
                self.best_fitness = neighbor_fitness
                self.iterations_without_improvement = 0
            else:
                self.iterations_without_improvement += 1

            if self.best_fitness == 1.0:
                break  # Exit if maximum fitness is reached

    def generate_neighbor(self, individual):
        """
        Generate a neighbor using Simulated Annealing.

        Args:
        - individual: The current individual.

        Returns:
        - neighbor: The generated neighbor.
        """
        # Simulated Annealing: Generate a neighbor by flipping bits with decreasing probability
        index_to_mutate = choices(range(self.genome_size), k=self.genome_size)
        neighbor = list(individual)

        for index in index_to_mutate:
            neighbor[index] = 1 - neighbor[index]

        # Probability of accepting a worse solution
        temperature = max(1.0, self.iterations_without_improvement)
        acceptance_probability = self.acceptance_probability(self.best_fitness, self.problem(neighbor), temperature)

        if random() < acceptance_probability:
            return neighbor
        else:
            return individual

    @staticmethod
    def acceptance_probability(current_fitness, new_fitness, temperature):
        """
        Calculate the acceptance probability for Simulated Annealing.

        Args:
        - current_fitness: The fitness of the current solution.
        - new_fitness: The fitness of the new solution.
        - temperature: The current temperature.

        Returns:
        - probability: The acceptance probability.
        """
        if new_fitness > current_fitness:
            return 1.0
        else:
            return exp((new_fitness - current_fitness) / temperature)


# Usage of the Genetic Algorithm
problem_instances = [1, 2, 5, 10]

for instance in problem_instances:
    fitness_value = instance
    fitness = lab9_lib.make_problem(fitness_value)
    fitness.calls_increment = fitness_value

    ga = GeneticAlgorithm(problem=fitness, genome_size=1000, population_size=50, generations=100, mutation_rate=0.1, local_search_iterations=10, convergence_threshold=5)
    best_solution = ga.run()

    print(f"Best solution for problem instance {instance}: {''.join(map(str, best_solution))}")
    print(f"Best fitness: {fitness(best_solution):.2%}")
    print(f"Total fitness calls: {fitness.calls}\n")

Best solution for problem instance 1: 01101011010011010001111001110001111100011000001101111001001100000010010011110000001010000100001101110111100010110110001000101101101100111000111011110111110010110110111100101101100111001110001111100011011111110101101011001110111101010111011101001010010011010100111101110111010111010110010101001010101011111100100101010100110100000110000101111011110111101100000011011100011011010010100110101110111011100100111011010000000111111110111000101010011011100111011111111111111100111001110011010110111100011001000110000100001101110011110000011111000111011011001010010110101100101011101110000001101110000101110110000001000011010110100111011001111111110111111001111101110000111101110110110100100010011101111110001101110110100001111011110011000100000101010011011010111011000100011110110011001011101111111011001111111010111100100010100111001101000101101101011101101000010110000001010010111011100101010101100110000111000001100100010011110111000100010000100000101111111101111001